In [1]:
%pip install --upgrade firebase-admin

     |████████████████████████████████| 112kB 6.9MB/s 
  Found existing installation: firebase-admin 4.1.0
    Uninstalling firebase-admin-4.1.0:
      Successfully uninstalled firebase-admin-4.1.0


In [3]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate('colab-service-account.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [4]:
posts_ref = db.collection(u'posts')

query = posts_ref.where(u'property', u'==', u'lad-000000003-001'
    ).where(u'refHash', u'==', u'#e0yxeeaf596p8p1tvskqr'
    ).limit(100)

results = query.stream()
records = {}
for doc in results:
    print(f'{doc.id} => {doc.to_dict()}')
    records[doc.id] = doc.to_dict()

print(len(records))

04v0QsvdM0RZbuiFEqgk => {'doNotTrack': None, 'refHash': '#e0yxeeaf596p8p1tvskqr', 'width': 1440, 'timeZoneOffset': -60, 'userHash': 'tn0kwtwq9z7y9a2ld75ut', 'referrer': 'https://ladder.io/', 'cookieEnabled': True, 'userAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 'href': 'https://try.ladder.io/?utm_expid=.OShV8i84QBebyz05Qhs27g.1&utm_referrer=#tn0kwtwq9z7y9a2ld75ut', 'locale': 'en-US', 'colorDepth': 24, 'height': 900, 'property': 'lad-000000003-001', 'title': 'Award-Winning Marketing Agency: Growth, Scale, & ROI - Ladder', 'timeZone': 'Europe/London'}
0OgThOnGGrPRoVGWVE0e => {'href': 'https://ladder.io/playbook?_ga=2.254068552.1128485427.1592602400-2128290037.1590599767#c56jba69bvac4iebsbrhje', 'locale': 'en-US', 'colorDepth': 24, 'property': 'lad-000000003-001', 'height': 1050, 'title': 'Ladder.io - Playbook', 'timeZone': 'America/Los_Angeles', 'doNotTrack': '1', 'refHash': '#e0yxeeaf596p8p1tvskqr',

In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

def parse_var(var, request):
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and var in request_json:
        return request_json[var]
    elif request_args and var in request_args:
        return request_args[var]
    else:
        return None

def main(request):
    """
    For testing input
    {"property": "lad-000000003-001", "refHash": "#e0yxeeaf596p8p1tvskqr"}
    """
    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows GET requests from any origin with the Content-Type
        # header and caches preflight response for an 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'POST',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    prop_id = parse_var("property", request)
    refHash = parse_var("refHash", request)

    firebase_admin.initialize_app()
    db = firestore.client()

    posts_ref = db.collection(u'posts')

    query = posts_ref.where(
        u'property', u'==', u'{}'.format(prop_id)).where(
            u'refHash', u'==', u'{}'.format(refHash)).limit(100)

    results = query.stream()
    records = []
    for doc in results:
        records.append(doc.id)

    share_count = len(records)

    # Set CORS headers for the main request
    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    # Return a 200 status
    return (str(share_count), 200, headers)